In [1]:
import duckdb
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
dotenv_path = Path("/Users/arthurgonzaga/DataKick/.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:

# Configuração das credenciais do MinIO
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
MINIO_ENDPOINT = "localhost:9000"

In [4]:
# Conectando ao DuckDB
con = duckdb.connect()

config = f"""
INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='{MINIO_ACCESS_KEY}';
SET s3_secret_access_key='{MINIO_SECRET_KEY}';
SET s3_endpoint='{MINIO_ENDPOINT}'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO
"""

# Configurando as credenciais e o endpoint do MinIO no DuckDB
con.execute(config)

In [5]:
print(config)


INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='jknVRF5166kyQRLiK7gc';
SET s3_secret_access_key='OKczJuafJxBfAQ8xn9hiHguhbQtQgmYALDwLY1Sy';
SET s3_endpoint='localhost:9000'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO



# Matches

In [6]:
minio_bronze_path = "s3://bronze/api_futebol/campeonato_brasileiro/*/*.json"

In [8]:
query = f'''
    SELECT
        CAST(partida_id AS INT) AS id_partida,
        campeonato['nome'] AS championship_name,
        campeonato['edicao']['temporada'] AS season,
        time_mandante['nome_popular'] AS home_team,
        time_visitante['nome_popular'] AS away_team,
        placar_mandante AS home_score,
        placar_visitante AS away_score,
        status,
        data_realizacao_iso AS dt_game_day,
        REPLACE(estatisticas.mandante.posse_de_bola, '"', '') AS home_ball_possession,
        estatisticas.mandante.escanteios AS home_corners,
        estatisticas.mandante.impedimentos AS home_offside,
        estatisticas.mandante.passes.total AS home_passes,
        estatisticas.mandante.passes.completos AS home_passes_completed,
        estatisticas.mandante.passes.errados AS home_passes_failed,
        estatisticas.mandante.finalizacao.total AS home_shoots,
        estatisticas.mandante.finalizacao.no_gol AS home_shoots_on_goal,
        estatisticas.mandante.finalizacao.pra_fora AS home_shoots_off_goal,
        estatisticas.mandante.finalizacao.na_trave AS home_shoots_on_crossbar,
        estatisticas.mandante.finalizacao.bloqueado AS home_shoots_blocked,
        estatisticas.mandante.defensivo.defesas AS home_defenses,
        estatisticas.mandante.desarmes AS home_disarms,
        escalacoes.mandante.esquema_tatico AS home_tactical_schema,
        REPLACE(estatisticas.visitante.posse_de_bola, '"', '') AS away_ball_possession,
        estatisticas.visitante.escanteios AS away_corners,
        estatisticas.visitante.impedimentos AS away_offside,
        estatisticas.visitante.passes.total AS away_passes,
        estatisticas.visitante.passes.completos AS away_passes_completed,
        estatisticas.visitante.passes.errados AS away_passes_failed,
        estatisticas.visitante.finalizacao.total AS away_shoots,
        estatisticas.visitante.finalizacao.no_gol AS away_shoots_on_goal,
        estatisticas.visitante.finalizacao.pra_fora AS away_shoots_off_goal,
        estatisticas.visitante.finalizacao.na_trave AS away_shoots_on_crossbar,
        estatisticas.visitante.finalizacao.bloqueado AS away_shoots_blocked,
        estatisticas.visitante.defensivo.defesas AS away_defenses,
        estatisticas.visitante.desarmes AS away_disarms,
        escalacoes.visitante.esquema_tatico AS away_tactical_schema
    FROM read_json_auto('{minio_bronze_path}', union_by_name=True) AS t;
'''

In [9]:
# Carregando o CSV do MinIO para uma tabela temporária e lendo os dados
try:
    result = con.execute(query).df()
except Exception as e:
    print("Erro ao acessar o MinIO:", e)

result

,id_partida,championship_name,season,home_team,away_team,home_score,away_score,status,dt_game_day,home_ball_possession,...,away_passes_completed,away_passes_failed,away_shoots,away_shoots_on_goal,away_shoots_off_goal,away_shoots_on_crossbar,away_shoots_blocked,away_defenses,away_disarms,away_tactical_schema
0,17037.0,Campeonato Brasileiro,2024,Internacional,Corinthians,1.0,0.0,finalizado,2024-06-19T21:30:00-0300,49%,...,317,79,9,3,6,0,4,6,14,4-3-3
1,17039.0,Campeonato Brasileiro,2024,Cruzeiro,Fluminense,2.0,0.0,finalizado,2024-06-19T21:30:00-0300,40%,...,511,60,9,4,5,0,0,6,7,4-3-3
2,17041.0,Campeonato Brasileiro,2024,Vitória,Atlético-MG,4.0,2.0,finalizado,2024-06-20T18:30:00-0300,44%,...,469,78,13,7,6,1,5,2,16,4-5-1
3,17043.0,Campeonato Brasileiro,2024,Flamengo,Bahia,2.0,1.0,finalizado,2024-06-20T20:00:00-0300,43%,...,662,53,8,3,5,0,0,5,8,4-4-2
4,17045.0,Campeonato Brasileiro,2024,Botafogo,Athletico-PR,1.0,1.0,finalizado,2024-06-19T19:00:00-0300,60%,...,257,59,12,7,5,0,2,1,10,4-3-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,17028.0,Campeonato Brasileiro,2024,Corinthians,São Paulo,2.0,2.0,finalizado,2024-06-16T16:00:00-0300,43%,...,465,71,9,8,1,0,3,8,17,4-5-1
266,17030.0,Campeonato Brasileiro,2024,Bragantino,Juventude,2.0,1.0,finalizado,2024-06-15T18:30:00-0300,44%,...,477,58,9,6,3,1,4,3,16,4-3-3
267,17031.0,Campeonato Brasileiro,2024,Athletico-PR,Flamengo,1.0,1.0,finalizado,2024-06-16T16:00:00-0300,48%,...,336,84,8,6,2,0,0,6,8,4-3-3
268,17032.0,Campeonato Brasileiro,2024,Cuiabá,Fortaleza,5.0,0.0,finalizado,2024-06-16T18:30:00-0300,53%,...,337,59,2,0,2,0,0,5,10,4-3-3


In [11]:
# Fechar a conexão
con.close()